In [ ]:
import io
import json
import logging
import os
import sys
from datetime import datetime
from pathlib import Path

import jinja2
import msal
import requests
from dotenv import load_dotenv
from O365 import Account

load_dotenv(Path("~").expanduser() / "O365.env")

In [ ]:
class GraphApi:
    def __init__(self, config):
        self.config = config
        self.client = self.getGraphClient()

    def getGraphClient(self):
        # Create a preferably long-lived app instance which maintains a token cache.
        app = msal.ConfidentialClientApplication(
            self.config["client_id"],
            authority=self.config["authority"],
            client_credential=self.config["client_secret"],
            # token_cache=...  # Default cache is in memory only.
            # You can learn how to use SerializableTokenCache from
            # https://msal-python.rtfd.io/en/latest/#msal.SerializableTokenCache
        )

        # The pattern to acquire a token looks like this.
        result = None

        # Firstly, looks up a token from cache
        # Since we are looking for token for the current app, NOT for an end user,
        # notice we give account parameter as None.
        auth_result = app.acquire_token_silent(config["scope"], account=None)

        if not result:
            auth_result = app.acquire_token_for_client(scopes=config["scope"])

        if not "access_token" in auth_result:
            print(auth_result.get("error"))
            print(auth_result.get("error_description"))
            print(
                auth_result.get("correlation_id")
            )  # You may need this when reporting a bug
            # Calling graph using the access token
            return False
        return auth_result

In [ ]:
# Optional logging
logging.basicConfig(level=logging.DEBUG)

config = {
    "client_id": os.environ["O365_client_id"],
    "client_secret": os.environ["O365_client_secret"],
    "authority": f"https://login.microsoftonline.com/{os.environ['O365_tenant']}",
    "scope": ["https://graph.microsoft.com/.default"],
    "endpoint": "https://graph.microsoft.com/v1.0/",
}
auth_result = GraphApi(config)